# Push/Pull to AWS
* Training DF files
* then push other data

In [13]:
import pandas as pd
import boto3
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm_notebook
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")
#set path directory

#load access key
HOME = os.path.expanduser('~')
KEYPATH = "SWEMLv2.0/AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'national-snow-model'
BUCKET = S3.Bucket(BUCKET_NAME)



Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



## Training DF regional files

In [42]:
#Upload nearest-snotel dictionaries
regionlist = ['SouthernRockies', 'Southwest', 'Northwest']
output_res = 300

for region in regionlist:
    print(region)
    head_folder = f"SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution/"
    head_folder_dir = f"{HOME}/{head_folder}"

    #Get files only
    files = [f for f in listdir(head_folder_dir) if isfile(join(head_folder_dir, f))]
    for file in tqdm_notebook(files):
        S3.meta.client.upload_file(Filename= f"{head_folder_dir}{file}", Bucket=BUCKET_NAME, Key=f"{head_folder}{file}")

    #get list of directories to download
    dirs = [ f.path for f in os.scandir(head_folder_dir) if f.is_dir() ]
    for dir in dirs:
        print(f"Folder name: {dir.split('300M_Resolution/')[-1]}")
        awsfolderpath = f"SWEMLv2.0{dir.split('SWEMLv2.0')[-1]}"
        headfolderpath = f"{HOME}/{awsfolderpath}"
        files = [f for f in listdir(headfolderpath) if isfile(join(headfolderpath, f))]
        for file in tqdm_notebook(files):
            S3.meta.client.upload_file(Filename= f"{headfolderpath}/{file}", Bucket=BUCKET_NAME, Key=f"{awsfolderpath}/{file}")

        #Get subfolders
        subdirs = [ f.path for f in os.scandir(headfolderpath) if f.is_dir() ]
        for dir in subdirs:
            print(f"Folder name: {dir.split('300M_Resolution/')[-1]}")
            awsfolderpath2 = f"SWEMLv2.0{dir.split('SWEMLv2.0')[-1]}"
            headfolderpath2 = f"{HOME}/{awsfolderpath2}"
            files = [f for f in listdir(headfolderpath2) if isfile(join(headfolderpath2, f))]
            for file in tqdm_notebook(files):
                S3.meta.client.upload_file(Filename= f"{headfolderpath2}/{file}", Bucket=BUCKET_NAME, Key=f"{awsfolderpath2}/{file}")



        




SouthernRockies


  0%|          | 0/3 [00:00<?, ?it/s]

Folder name: Obsdf


  0%|          | 0/14 [00:00<?, ?it/s]

Folder name: GeoObsDFs


...

0it [00:00, ?it/s]

Folder name: Seasonality_PrecipVIIRSGeoObsDFs


0it [00:00, ?it/s]

Folder name: Seasonality_PrecipVIIRSGeoObsDFs/20_fSCA_Thresh


  0%|          | 0/2 [00:00<?, ?it/s]

## Get files

In [45]:
#Get data
for region in regionlist:
    path = f"SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution/"

    #Make directory if it does not exist
    if not os.path.exists(f"{HOME}/{path}"):
        print("Path not present, making")
        os.makedirs(path, exist_ok=True)
    
    #Identify the potential files to download
    files = [objects.key for objects in BUCKET.objects.filter(Prefix=f"SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution")]
    
    for file in files:
        #check to see if the file is there
        if os.path.exists(f"{HOME}/{file}") == False:
            print('Downloading file')
            S3.Bucket(BUCKET_NAME).download_file(file, f"{path}/{file}")

        else:
            print('File already present...')

File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already pre